In [178]:
from math import sqrt
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm

In [179]:
import numpy as np
from scipy.stats import norm

S0 = 100
K = 104
T = 1
num_steps = 10
sigma = 0.2
mu = 0.08
r = 0.04
n_path = 10_000


def gbm(S0, r, sigma, t, n_path):
    drift = r * t
    vol = sigma * np.sqrt(t) * np.random.standard_normal((n_path, num_steps+1))
    exponent = drift + vol
    paths = S0 * np.exp(exponent)
    return paths

t = np.linspace(0, T, num_steps + 1)
S = gbm(S0, r, sigma, t, n_path)
S

array([[100.        ,  85.12199435, 101.46517713, ..., 126.30966334,
         92.91854242,  80.19202072],
       [100.        ,  97.35269051,  97.47003726, ...,  78.89405737,
         94.22718536, 103.35490023],
       [100.        , 101.16517989, 102.83425436, ...,  76.50291651,
         61.03808046, 109.09671826],
       ...,
       [100.        , 105.70069656, 110.48168257, ...,  83.69960489,
         83.87288974, 110.19019098],
       [100.        ,  96.27184994,  97.69464736, ...,  89.23117757,
        109.10905959, 116.35204503],
       [100.        ,  96.04690096,  79.29512693, ...,  76.77226433,
        128.67640157, 115.1324382 ]])

In [180]:
def _d1(S, K, r, sigma, t, T):
    t2m = T - t
    numerator = np.log(S / K) + (r + 0.5 * sigma**2) * t2m
    denominator = sigma * np.sqrt(t2m)
    with np.errstate(divide='ignore'):
        return numerator / denominator

def _d2(d1, sigma, t, T):
    t2m = T - t
    return d1 - sigma * np.sqrt(t2m)

def call(S, K, r, t, T, d1, d2):
    return S * norm.cdf(d1) - K * np.exp(-r * (T - t)) * norm.cdf(d2)

def put(S, K, r, t, T, d1, d2):
    return -S * norm.cdf(-d1) + K * np.exp(-r * (T - t)) * norm.cdf(-d2)

d1 = _d1(S, K, r, sigma, t, T)
d2 = _d2(d1, sigma, t, T)

c = call(S, K, r, t, T, d1, d2)
p = put(S, K, r, t, T, d1, d2)
p[0, 0]

7.923576011963995